In [1]:
# Install the required libraries
!pip install requests pandas


In [1]:
import time
import requests
import pandas as pd


In [2]:
YOUR_GITHUB_TOKEN = "GITHUB TOKEN"  # Replace with your actual GitHub token
BASE_URL = "https://api.github.com"

def make_request(url):
    headers = {'Authorization': f'token {YOUR_GITHUB_TOKEN}'}
    for attempt in range(3):  # Retry mechanism
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise error for bad status
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying...")
            time.sleep(2)
    print(f"Failed to fetch data from {url} after 3 attempts.")
    return {}



In [3]:
def get_users_in_chennai(min_followers=50):
    url = f"{BASE_URL}/search/users?q=location:Chennai+followers:>{min_followers}&per_page=100"
    users = []
    while url:
        data = make_request(url)
        if not data or 'items' not in data:
            break
        users.extend(data['items'])
        url = data.get('next_url', None)
        time.sleep(1)  # Delay to avoid rate limiting
    return users

In [4]:
def get_user_details(login):
    url = f"{BASE_URL}/users/{login}"
    user_data = make_request(url)
    def safe_strip(value):
        return value.strip() if isinstance(value, str) else ""
    return {
        "login": user_data.get("login", ""),
        "name": safe_strip(user_data.get("name", "")),
        "company": clean_company(user_data.get("company", "")),
        "location": safe_strip(user_data.get("location", "")),
        "email": safe_strip(user_data.get("email", "")),
        "hireable": user_data.get("hireable", False),
        "bio": safe_strip(user_data.get("bio", "")),
        "public_repos": user_data.get("public_repos", 0),
        "followers": user_data.get("followers", 0),
        "following": user_data.get("following", 0),
        "created_at": safe_strip(user_data.get("created_at", ""))
    }

In [5]:
def clean_company(company):
    return company.strip('@').strip().upper() if company else "Unknown"



In [6]:
def get_user_repositories(login, max_repos=500):
    repos = []
    url = f"{BASE_URL}/users/{login}/repos?per_page=100"
    while url and len(repos) < max_repos:
        data = make_request(url)
        if not data:
            break
        for repo in data:
            # Check if the license is not None before trying to access "key"
            license_name = repo["license"]["key"] if repo.get("license") else ""
            repos.append({
                "login": login,
                "full_name": repo["full_name"],
                "created_at": repo["created_at"],
                "stargazers_count": repo["stargazers_count"],
                "watchers_count": repo["watchers_count"],
                "language": repo.get("language", ""),
                "has_projects": repo.get("has_projects", False),
                "has_wiki": repo.get("has_wiki", False),
                "license_name": license_name
            })
        if len(data) < 100:  # Stop if less than 100 results (end of repos)
            break
        time.sleep(1)  # Delay to avoid rate limiting
    return repos



In [7]:
# Main code to gather and save data
users = get_users_in_chennai()
user_data = []
repo_data = []

for user in users:
    user_details = get_user_details(user['login'])
    user_data.append(user_details)

    user_repos = get_user_repositories(user['login'])
    repo_data.extend(user_repos)

# Save user data to CSV
user_df = pd.DataFrame(user_data)
user_df.to_csv('users.csv', index=False)

# Save repository data to CSV
repo_df = pd.DataFrame(repo_data)
repo_df.to_csv('repositories.csv', index=False)

print("Data saved to users.csv and repositories.csv")

Data saved to users.csv and repositories.csv
